# Structural Overview of CDK2 (PDB ID: 1H1Q)

- The 1H1Q structure represents the **active form of human Cyclin-Dependent Kinase 2 (CDK2)** bound to **Cyclin A** and the purine-based inhibitor **NU6094 (2A6)**.  
- CDK2 is a serine/threonine kinase that regulates the **G1/S phase transition** of the cell cycle, requiring both **Cyclin A binding** and **Thr160 phosphorylation** for full activation.  

- Structurally, CDK2 adopts the **typical kinase fold** with a small N-terminal lobe and a larger C-terminal lobe forming the **ATP-binding cleft**.  
- **Cyclin A binding** stabilizes the PSTAIRE helix and correctly positions the activation loop (T-loop) containing phosphorylated Thr160, enabling substrate and inhibitor binding.  
- The **inhibitor NU6094** binds in the ATP pocket, forming **key hydrogen bonds with hinge residues Glu81 and Leu83**, and engages in **hydrophobic interactions with Phe80, Phe82, and Asp86**—stabilizing the active site and mimicking ATP binding.  
- Later analogs (e.g., NU6102) introduced a sulfonamide group to form additional hydrogen bonds, improving affinity by nearly 1000-fold.  

- For receptor preparation, only **chains A and B** (CDK2–Cyclin A complex) will be retained, excluding all water molecules and heteroatoms.  

The structure was solved at **2.5 Å resolution**, capturing a fully active, phosphorylated CDK2/Cyclin A complex with a bound inhibitor—making it an ideal model for **virtual screening and docking studies**.

In [1]:
import os
import subprocess
import time
import prody
from prody import parsePDB, writePDB, calcCenter

# --- SETTINGS ---
pdb_id = "1H1Q"
box_size = (20.0, 20.0, 20.0)
ligand_resname = "2A6"  # co-crystallized ligand in 1H1Q

/home/pchatterjee/.local/lib/python3.10/site-packages/prody/utilities/misctools.py:424: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


### **Step 1 : Download the PDB Structure**
The CDK2 structure (PDB **1H1Q**) is downloaded directly from the RCSB Protein Data Bank.

In [2]:
print(f"Downloading {pdb_id} from RCSB...")
os.system(f"curl -s https://files.rcsb.org/view/{pdb_id}.pdb -o {pdb_id}.pdb")

0

### **Step 2 : Select Protein Chains A and B**
Water molecules and heteroatoms (ligands, ions) are removed, keeping only the receptor atoms.

In [3]:
print("Selecting receptor atoms (chains A and B, excluding water/heteroatoms)...")
atoms = parsePDB(pdb_id)
receptor_atoms = atoms.select("(chain A or chain B) and not water and not hetero")
if receptor_atoms is None:
    raise ValueError("No receptor atoms found. Check chain IDs or PDB content.")
writePDB(f"{pdb_id}_receptor_atoms.pdb", receptor_atoms)

@> Connecting wwPDB FTP server RCSB PDB (USA).
@> Downloading PDB files via FTP failed, trying HTTP.


Selecting receptor atoms (chains A and B, excluding water/heteroatoms)...


@> 1h1q downloaded (1h1q.pdb.gz)
@> PDB download via HTTP completed (1 downloaded, 0 failed).
@> 9325 atoms and 1 coordinate set(s) were parsed in 0.04s.


'1H1Q_receptor_atoms.pdb'

### **Step 3 : Define the Docking Box**
The geometric center of the co-crystal ligand (`2A6`) is used to define the center of the docking grid box.

In [4]:
print(f"Calculating box center using co-ligand: {ligand_resname}")
ligand_atoms = atoms.select(f"chain A and resname {ligand_resname}")
if ligand_atoms is None:
    raise ValueError(f"No ligand found with resname '{ligand_resname}' in chain A.")
cx, cy, cz = calcCenter(ligand_atoms)
print(f"Box center = ({cx:.3f}, {cy:.3f}, {cz:.3f})")

Calculating box center using co-ligand: 2A6
Box center = (6.145, 44.176, 50.828)


### **Step 4 : Prepare the Receptor with Meeko**
Meeko converts the cleaned PDB into a **PDBQT** format compatible with AutoDock Vina and defines the docking grid box.

This includes:
- Adding hydrogens  
- Assigning Gasteiger charges  
- Creating `.pdbqt` and `.box.pdb` files

##### **Note on File Handling in Jupyter**

In some cases, `writePDB()` from ProDy may not immediately create the output file if:
- the atom selection (`receptor_atoms`) is empty or invalid (e.g., wrong chain IDs), or  
- the Jupyter environment hasn’t fully flushed the file to disk when `os.rename()` is called.

If you encounter a `FileNotFoundError`, first check:
```python
print([f for f in os.listdir('.') if f.endswith('.pdb')])

In [5]:
out_base = f"{pdb_id}_receptorH"
renamed_pdb = "receptor_1H1Q.pdb"

# --- Wait for file to appear before renaming ---
pdb_path = f"{pdb_id}_receptor_atoms.pdb"
for _ in range(10):
    if os.path.exists(pdb_path):
        break
    time.sleep(0.2)  # wait 200 ms
else:
    raise FileNotFoundError(f"{pdb_path} not found after writing.")

# --- Now safe to rename ---
os.rename(pdb_path, renamed_pdb)
print("Preparing receptor with Meeko...")

subprocess.run([
    "mk_prepare_receptor.py",
    "-i", renamed_pdb,
    "-o", out_base,
    "-p", "-v",
    "--box_center", str(cx), str(cy), str(cz),
    "--box_size", str(box_size[0]), str(box_size[1]), str(box_size[2])
], check=True)

print(f"Receptor prepared successfully → {out_base}.pdbqt and {out_base}.box.pdb")


Preparing receptor with Meeko...


@> 4471 atoms and 1 coordinate set(s) were parsed in 0.02s.



Files written:
  1H1Q_receptorH.pdbqt <-- static (i.e., rigid) receptor input file
1H1Q_receptorH.box.txt <-- Vina-style box dimension file
1H1Q_receptorH.box.pdb <-- PDB file to visualize the grid box
Receptor prepared successfully → 1H1Q_receptorH.pdbqt and 1H1Q_receptorH.box.pdb
